In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymc3 as pm

%matplotlib inline

In [ ]:
df = pd.read_csv('datasets/mlb_2013-2016.csv')
df.head()

In [ ]:
import seaborn as sns

In [ ]:
# df.plot(kind='scatter', x='Season', y='Team Salary')
sns.violinplot(x='Season', y='Team Salary', data=df)

In [ ]:
df.columns

In [ ]:
# Let's predict 'Winning %' from the rest of the columns.
cols = list(df.columns)
cols.remove('Season')
cols.remove('Team')
cols.remove('Team Salary (in millions)')
cols.remove('League')
cols.remove('Wins')
cols.remove('Losses')

feat_cols = list(set(cols) - set(['Winning %']))
y_cols = ['Winning %']

In [ ]:
feat_cols

In [ ]:
import theano.tensor as tt
with pm.Model() as model:
    weights = pm.Normal('weights', mu=0, sd=100**2, shape=(len(feat_cols),))
    perc_losses = tt.dot(weights, df[feat_cols].T)
    alpha = 1
    beta = 1 / perc_losses
    like = pm.Beta('likelihood', alpha=alpha, beta=beta, observed=df[y_cols])

In [ ]:
with model:
    trace = pm.sample(2000)

In [ ]:
pm.traceplot(trace)